## Week 5 Report
#### Exploring Neighborhoods of Hyderabad, India – To decide on ideal location for Food Court
##### a. Develop a dataframe having neighborhoods of Hyderabad, India by web scraping the data from Wikipedia page
##### b Derive geographical coordinates of the neighborhoods using Geocoder
##### c Use Foursquare API to obtain neighborhood venue data
##### d Derive the clusters, explore and study the same
##### e Decide and suggest best suitable cluster for building Food Court

#### Step 1- Import Libraries

In [15]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


#### Step 2- Data scarping from Wikipedia to Dataframe

In [16]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Hyderabad,_India").text

In [17]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [18]:
# create a list to store neighborhood data
neighborhoodList = []

In [19]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [20]:
# create a new DataFrame from the list
Hyd_df = pd.DataFrame({"Neighborhood": neighborhoodList})

Hyd_df.head()

,Neighborhood
0,A. C. Guards
1,A. S. Rao Nagar
2,Abhyudaya Nagar
3,Abids
4,Adibatla


In [21]:
# print the number of rows of the dataframe
Hyd_df.shape

(200, 1)

#### Step 3- Fetching Geographicak coordinate using Geocoder

In [22]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Hyderabad, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [23]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in Hyd_df["Neighborhood"].tolist() ]
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
Hyd_df['Latitude'] = df_coords['Latitude']
Hyd_df['Longitude'] = df_coords['Longitude']

# check the neighborhoods and the coordinates
print(Hyd_df.shape)
Hyd_df

(200, 3)


,Neighborhood,Latitude,Longitude
0,A. C. Guards,17.395015,78.459812
1,A. S. Rao Nagar,17.411200,78.508240
2,Abhyudaya Nagar,17.337650,78.564140
3,Abids,17.389800,78.476580
4,Adibatla,17.235790,78.541300
5,Adikmet,17.410610,78.515130
6,Afzal Gunj,17.377510,78.480050
7,Aghapura,17.389165,78.463855
8,"Aliabad, Hyderabad",17.342590,78.476260
9,Alijah Kotla,17.360680,78.479980


In [25]:
# save the DataFrame as CSV file
Hyd_df.to_csv("Hyd_df.csv", index=False)

#### Step 4- Create map of Hyderabad, India with Neighborhood superimposed

In [26]:
address = 'Hyderbad, India'

geolocator = Nominatim(user_agent="Hyd_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hyderabad are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Hyderabad are 17.3715198, 78.5114269.


In [27]:
# create map of Hyderabad using latitude and longitude values
map_Hyd = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(Hyd_df['Latitude'], Hyd_df['Longitude'], Hyd_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Hyd)  
    
map_Hyd

#### Step 5- Exploring Neighborhood using Foursquare - top 100 venues within 2km radius

In [28]:
# define Foursquare Credentials and Version
CLIENT_ID = 'QZNZEU5PABBP2CJULJPLONNLW4MZE1VBJ5HWRFUZCMWT1SL3' # your Foursquare ID
CLIENT_SECRET = 'O1KCKOLBUDFF22CDDRONNJHTBFDTBAESXZZNS5JMOYPYXLYF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QZNZEU5PABBP2CJULJPLONNLW4MZE1VBJ5HWRFUZCMWT1SL3
CLIENT_SECRET:O1KCKOLBUDFF22CDDRONNJHTBFDTBAESXZZNS5JMOYPYXLYF


In [29]:
### Now, let's get the top 100 venues that are within a radius of 2000 meters.
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(Hyd_df['Latitude'], Hyd_df['Longitude'], Hyd_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [30]:
# convert the venues list into a new DataFrame
Hyd_new_df = pd.DataFrame(venues)

# define the column names
Hyd_new_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(Hyd_new_df.shape)
Hyd_new_df.head()

(5567, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,A. C. Guards,17.395015,78.459812,Subhan Bakery,17.392412,78.464712,Bakery
1,A. C. Guards,17.395015,78.459812,Chicha's,17.403255,78.460152,Hyderabadi Restaurant
2,A. C. Guards,17.395015,78.459812,Jewel Of Nizam,17.403869,78.461194,Middle Eastern Restaurant
3,A. C. Guards,17.395015,78.459812,Nizam club,17.403221,78.468729,Lounge
4,A. C. Guards,17.395015,78.459812,Laxman Ki Bandi,17.378895,78.463973,South Indian Restaurant


In [31]:
print('There are {} uniques categories.'.format(len(Hyd_new_df['VenueCategory'].unique())))

There are 173 uniques categories.


In [32]:
# print out the list of categories
Hyd_new_df['VenueCategory'].unique()

array(['Bakery', 'Hyderabadi Restaurant', 'Middle Eastern Restaurant',
       'Lounge', 'South Indian Restaurant', 'Café', 'Science Museum',
       'Bistro', 'Shoe Store', 'Indian Restaurant', 'Ice Cream Shop',
       'Diner', 'Neighborhood', 'Dessert Shop', 'Department Store',
       'Stadium', 'Coffee Shop', 'Snack Place', 'Performing Arts Venue',
       'Fast Food Restaurant', 'Hotel Bar', 'Hotel', 'Chinese Restaurant',
       'Mobile Phone Shop', 'Electronics Store', 'Shopping Mall',
       'Pizza Place', 'Clothing Store', 'Bus Station', 'Park',
       'Bookstore', 'Planetarium', 'Gift Shop', 'Movie Theater',
       'Sandwich Place', 'Convenience Store', 'Platform', 'Train Station',
       'Food Court', 'Restaurant', 'Juice Bar', 'Burger Joint',
       'Food Truck', 'Chaat Place', 'Breakfast Spot', 'Smoke Shop',
       'Gaming Cafe', 'Multiplex', 'Food', 'Tea Room', 'Garden Center',
       'History Museum', 'Monument / Landmark', 'Cheese Shop', 'Resort',
       'Italian Restaurant'

#### Step 6- Analyzing Neighborhood

In [33]:
# one hot encoding
Hyd_onehot = pd.get_dummies(Hyd_new_df[['VenueCategory']],prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Hyd_onehot['Neighborhoods'] = Hyd_new_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Hyd_onehot.columns[-1]] + list(Hyd_onehot.columns[:-1])
Hyd_onehot = Hyd_onehot[fixed_columns]

print(Hyd_onehot.shape)
Hyd_onehot.head()

(5567, 174)


,Neighborhoods,ATM,Afghan Restaurant,Airport Service,American Restaurant,Andhra Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Bus Station,Business Service,Cable Car,Cafeteria,Café,Campground,Candy Store,Castle,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Hunan Restaurant,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Jewelry Store,Juice Bar,Kitchen Supply Store,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Neighborhood,Night Market,Nightclub,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pie Shop,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Rajasthani Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Wings Joint,Women's Store,Zoo
0,A. C. Guards,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,A. C. Guards,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,A. C. Guards,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,A. C. Guards,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,A. C. Guards,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [34]:
Hyd_grouped = Hyd_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(Hyd_grouped.shape)
Hyd_grouped

(196, 174)


,Neighborhoods,ATM,Afghan Restaurant,Airport Service,American Restaurant,Andhra Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Bus Station,Business Service,Cable Car,Cafeteria,Café,Campground,Candy Store,Castle,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Hunan Restaurant,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Jewelry Store,Juice Bar,Kitchen Supply Store,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Neighborhood,Night Market,Nightclub,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pie Shop,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Rajasthani Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Wings Joint,Women's Store,Zoo
0,A. C. Guards,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.083333,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.033333,0.000000,0.016667,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.016667,0.016667,0.000000,0.00,0.00,0.033333,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.033333,0.000000,0.000000,0.016667,0.016667,0.150000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.016667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.016667,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.00,0.000000,0.016667,0.000000,0.016667,0.000000,0.00,0.033333,0.016667,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.016667,0.000000,0.016667,0.000000,0.016667,0.000000,0.033333,0.000000,0.000000,0.000000,0.0,0.016667,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,A. S. Rao Nagar,0.000000,0.000000,0.000000,0.000000,0.

In [35]:
len((Hyd_grouped[Hyd_grouped[("Food Court")] > 0]))
Hyd_Food_Court = Hyd_grouped[["Neighborhoods","Food Court"]]
print(Hyd_Food_Court.shape)
Hyd_Food_Court.head()

(196, 2)


,Neighborhoods,Food Court
0,A. C. Guards,0.000000
1,A. S. Rao Nagar,0.045455
2,Abhyudaya Nagar,0.000000
3,Abids,0.013889
4,Adikmet,0.047619


#### Step 7- Clusturing Neighborhood - run K-means to cluster neighborhoods into 5 clusters

In [36]:
# set number of clusters
kclusters = 5

kl_clustering = Hyd_Food_Court.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 0, 4, 1, 4, 0, 0, 3, 0], dtype=int32)

In [38]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
Hyd_Food_Court_merged = Hyd_Food_Court.copy()

# add clustering labels
Hyd_Food_Court_merged["Cluster Labels"] = kmeans.labels_

In [39]:
Hyd_Food_Court_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
Hyd_Food_Court_merged.head()

,Neighborhood,Food Court,Cluster Labels
0,A. C. Guards,0.000000,0
1,A. S. Rao Nagar,0.045455,1
2,Abhyudaya Nagar,0.000000,0
3,Abids,0.013889,4
4,Adikmet,0.047619,1


In [40]:
# merge Hyd_Food_court_grouped with Hyd_df to add latitude/longitude for each neighborhood

Hyd_Food_Court_final = pd.merge(Hyd_Food_Court_merged, Hyd_df)

print(Hyd_Food_Court_final.shape)
Hyd_Food_Court_final.head()

(196, 5)


,Neighborhood,Food Court,Cluster Labels,Latitude,Longitude
0,A. C. Guards,0.000000,0,17.395015,78.459812
1,A. S. Rao Nagar,0.045455,1,17.411200,78.508240
2,Abhyudaya Nagar,0.000000,0,17.337650,78.564140
3,Abids,0.013889,4,17.389800,78.476580
4,Adikmet,0.047619,1,17.410610,78.515130


In [41]:
# sort the results by Cluster Labels
print(Hyd_Food_Court_final.shape)
Hyd_Food_Court_final.sort_values(["Cluster Labels"], inplace=True)
Hyd_Food_Court_final

(196, 5)


,Neighborhood,Food Court,Cluster Labels,Latitude,Longitude
0,A. C. Guards,0.000000,0,17.395015,78.459812
112,Macha Bollaram,0.000000,0,17.525910,78.376330
113,Madannapet,0.000000,0,17.357880,78.501710
114,Madeenaguda,0.000000,0,17.495360,78.333360
116,"Madina building, Hyderabad",0.000000,0,17.453600,78.415320
118,Mahatma Gandhi Road (Secunderabad),0.000000,0,17.428497,78.487907
119,Malakpet,0.000000,0,17.374930,78.515670
120,Malkajgiri,0.000000,0,17.447370,78.535200
111,Lothkunta,0.000000,0,17.494050,78.515140
121,Malkajgiri mandal,0.000000,0,17.447370,78.535200


#### Step 8- Visulaizing the Cluster

In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Hyd_Food_Court_final['Latitude'], Hyd_Food_Court_final['Longitude'], Hyd_Food_Court_final['Neighborhood'], Hyd_Food_Court_final['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [45]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

#### Step 9- Analyzing the Clusters

In [46]:
Hyd_Food_Court_final.loc[Hyd_Food_Court_final['Cluster Labels'] == 0]

,Neighborhood,Food Court,Cluster Labels,Latitude,Longitude
0,A. C. Guards,0.0,0,17.395015,78.459812
112,Macha Bollaram,0.0,0,17.525910,78.376330
113,Madannapet,0.0,0,17.357880,78.501710
114,Madeenaguda,0.0,0,17.495360,78.333360
116,"Madina building, Hyderabad",0.0,0,17.453600,78.415320
118,Mahatma Gandhi Road (Secunderabad),0.0,0,17.428497,78.487907
119,Malakpet,0.0,0,17.374930,78.515670
120,Malkajgiri,0.0,0,17.447370,78.535200
111,Lothkunta,0.0,0,17.494050,78.515140
121,Malkajgiri mandal,0.0,0,17.447370,78.535200


In [47]:
Hyd_Food_Court_final.loc[Hyd_Food_Court_final['Cluster Labels'] == 1]

,Neighborhood,Food Court,Cluster Labels,Latitude,Longitude
1,A. S. Rao Nagar,0.045455,1,17.41120,78.50824
19,Azampura,0.058824,1,17.37272,78.49047
4,Adikmet,0.047619,1,17.41061,78.51513
176,"Ramnagar, Hyderabad",0.045455,1,17.41120,78.50824
159,Padmarao Nagar,0.052632,1,17.42616,78.50689
15,"Ashok Nagar, Hyderabad",0.033333,1,17.40784,78.49150
149,Nanakramguda,0.043478,1,17.41399,78.34580
156,Noorkhan Bazar,0.041667,1,17.37182,78.48544
31,Begum Bazaar,0.038462,1,17.37358,78.46921
157,Old Neredmet,0.058824,1,17.46259,78.51630


In [48]:
Hyd_Food_Court_final.loc[Hyd_Food_Court_final['Cluster Labels'] == 2]

,Neighborhood,Food Court,Cluster Labels,Latitude,Longitude
95,"Kings Colony, Shastripuram",0.25,2,17.32598,78.44735
85,Kakatiya Nagar,0.20,2,17.41099,78.61091


In [49]:
Hyd_Food_Court_final.loc[Hyd_Food_Court_final['Cluster Labels'] == 3]

,Neighborhood,Food Court,Cluster Labels,Latitude,Longitude
51,Dhoolpet,0.090909,3,17.37407,78.459910
49,Dabirpura,0.071429,3,17.36618,78.487360
104,Laad Bazaar,0.066667,3,17.36145,78.472090
46,Chilkalguda,0.071429,3,17.42675,78.512290
94,Khilwat,0.083333,3,17.35496,78.470290
128,"Mehboob ki Mehendi, Hyderabad",0.062500,3,17.36231,78.471027
66,Gulzar Houz,0.071429,3,17.36338,78.475020
192,Shah-Ali-Banda,0.071429,3,17.35739,78.473200
193,Shahran Market,0.071429,3,17.36489,78.476290
135,Moghalpura,0.071429,3,17.35881,78.477180


In [50]:
Hyd_Food_Court_final.loc[Hyd_Food_Court_final['Cluster Labels'] == 4]

,Neighborhood,Food Court,Cluster Labels,Latitude,Longitude
178,"Red Hills, Hyderabad",0.012821,4,17.397410,78.462750
34,Boggulkunta,0.014286,4,17.391370,78.480230
75,Hyderguda,0.012987,4,17.399220,78.480750
74,"Himayatnagar, Hyderabad",0.013333,4,17.404860,78.481210
184,Saifabad,0.024096,4,17.405390,78.471380
5,Afzal Gunj,0.026316,4,17.377510,78.480050
3,Abids,0.013889,4,17.389800,78.476580
84,Kachiguda,0.023256,4,17.386880,78.495530
190,Secunderabad,0.017857,4,17.442140,78.501930
92,Khairtabad,0.010000,4,17.405920,78.458560


#### Step 10- Concluding

#####	
The neighborhoods of Hyderabad is clustered in 5 Clusters, based on frequency of occurrence of Food Court.
1.	Cluster 0  (Red dots in map)- Neighborhood with zero or minimum number of Food court
2.	Cluster 1 (Violet dots in map)- Neighborhood with moderate presence of Food court
3.	Cluster 2 (Turquoise dots in map)- Neighborhood with limited number of Food court
4.	Cluster 3 (Green dots in map)-  Neighborhood with high density of Food court
5.	Cluster 4 (Amber dots in map)-- Neighborhood with moderate presence of Food court


Based on the this above depiction, it is observed there is scope of having good Food court Business in Cluster 0, followed by Cluster 2.
However this is an exploratory research, and there are other associated factors to take conclusion
